# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [4]:
from datetime import datetime as dt # importer la librairie qui permet de comparer les requêtes à la date du jour
from SPARQLWrapper import SPARQLWrapper, JSON # importer la librairie sparql

## Obtenir la liste des politiciens belges

In [55]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" # wikidata au format RDF
sparql = SPARQLWrapper(endpoint) # stockage de l'adressage dans une variable

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death

# stocker la requête dans une variable
# quels sont les uri's des politiciens belges, leur label/nom, leurs dates de naissance et de mort (s'ils sont morts) ?

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath 
WHERE 
{
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P106 wd:Q82955 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement) # adresse la requête à wikidata
sparql.setReturnFormat(JSON) # retourner le résultat au format json
results = sparql.query().convert() # stocker les résultats convertis dans une variable

rows = results['results']['bindings'] # créer une liste de résultats avec le lien associé à chaque label
print(f"\n{len(rows)} Belgian politicians found\n") # imprimer le nombre de politiciens recensés par la requête
print(rows[:10]) # imprimer les 10 premiers résultats


7657 Belgian politicians found

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17350686'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abdallah Kanfaoui'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1968-05-21T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13461801'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abdelaziz Charkaoui Hajri'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1951-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27903935'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abderrahim Lahlali'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1978-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3294738'}, 'person

## Filtrer pour n'afficher que les noms contenant "Bouchez"

In [31]:
name_filter = 'Bouchez' # créer une variable pour le filtre
max_results = 10 # limiter le résultat à 10 occurences

date_format = "%Y-%m-%dT%H:%M:%SZ" # définir le format à appliquer aux dates
filtered_rows = [row for row in rows if name_filter in row['personLabel']['value']] # appliquer le filtre et s'il existe, prendre la valeur

print(f"Displaying the first {max_results}:\n") # imprimer la phrase d'introduction des résultats (limités à 10 ici)

for row in filtered_rows[:max_results]: # pour que chaque ligne des 10 premiers résultats
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format) # stocker la valeur de la date de naissance au format date (string au départ)
        birth_year = birth_date.year # ne garder que l'année de la date de naissance
    except ValueError:
        birth_year = "????" # ignorer les valeurs inconnues
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format) # idem pour la date de mort
        death_year = death_date.year
    except ValueError: # unknown death date
        death_date = "????"
    except KeyError: # still alive
        death_date = "" # ignorer l'erreur si l'année de mort n'est pas encore connue
    print(f"{row['personLabel']['value']} ({birth_date}-{death_date})") # afficher la date complète en choississant la bonne variable

Displaying the first 10:

Georges-Louis Bouchez (1986-03-23 00:00:00-)


In [26]:
help(dt.strptime)

Help on built-in function strptime:

strptime(...) method of builtins.type instance
    string, format -> new datetime parsed from a string (like time.strptime()).



## Nouvelle requête pour les bulletins de la Ville de Bruxelles

In [91]:
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" # wikidata au format RDF
sparql = SPARQLWrapper(endpoint) # stockage de l'adressage dans une variable

# P39 = fonction occupée par une personne
# Q33126365 = bourgmestre de Bruxelles
# P580 = date de début
# P582 = date de fin

# stocker la requête dans une variable
# quels sont les uri's des politiciens belges, leur label/nom, leurs dates de naissance et de mort (s'ils sont morts) ?

statement = """
SELECT DISTINCT ?person ?personLabel ?start ?end
WHERE 
{
    ?person wdt:P39/wdt:P279* wd:Q33126365 .
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement) # adresse la requête à wikidata
sparql.setReturnFormat(JSON) # retourner le résultat au format json
results = sparql.query().convert() # stocker les résultats convertis dans une variable

rows = results['results']['bindings'] # créer une liste de résultats avec le lien associé à chaque label
print(f"\n{len(rows)} Bourgmestres de la Ville de Bruxelles trouvés\n") # imprimer le nombre de politiciens recensés par la requête

for row in rows:
    print(row['personLabel']['value']) # imprimer les 10 premiers résultats


47 Bourgmestres de la Ville de Bruxelles trouvés

Adolphe Max
Adrien de Grimberghe, seigneur d'Assche
André-Napoléon Fontainas
Antoine de Grimbergen van Assche
Charles Buls
Charles de Brouckère
Charles-Joseph, 4th Duke d'Ursel
Comte Guillaume de Merode, 5th Prince de Rubempré et d'Everberghe
Engelbert Taie
Felix Vanderstraeten
François-Jean Wyns de Raucourt
François-Xavier de Donnea
Freddy Thielemans
Frederick de Marselaer
Gabriel van Bemmel
Guillaume Van Volxem
Hendrik de Bloyere
Henri Joseph Van Langhenhoven
Henri Taye
Hervé Brouhon
Hyacinthe van der Fosse
Jacques Taye
Jan Grauls
Jan-Baptist Verlooy
Jean de Locquenghien
John II of Cottereau, Baron of Jauche
Joseph Van De Meulebroeck
Joseph van der Linden d'Hooghvorst
Jules Anspach
Jules Coelst
Lancelot I Schetz
Lancelot Schetz
Louis Devos
Louis de Wellens van ten Meulenberg
Lucien Cooremans
Marion Lemesre
Michel Demaret
Nicolas-Jean Rouppe
Norbert François Charles van Assche
Paul Arconati-Visconti
Philippe Close
Pierre Van Halteren


In [148]:
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" # wikidata au format RDF
sparql = SPARQLWrapper(endpoint) # stockage de l'adressage dans une variable

# P39 = fonction occupée par une personne
# Q33126365 = bourgmestre de Bruxelles
# P580 = date de début
# P582 = date de fin

statement = """
SELECT DISTINCT ?person ?personLabel ?fonctionLabel ?starttime ?endtime
WHERE 
{
    ?person wdt:P39 wd:Q33126365 .
    ?person p:P39 ?statement .
    
    OPTIONAL { ?statement ps:P39 ?fonction .}
    OPTIONAL { ?statement pq:P580 ?starttime .}
    OPTIONAL { ?statement pq:P582 ?endtime .}
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
ORDER BY ?starttime
"""

sparql.setQuery(statement) # adresse la requête à wikidata
sparql.setReturnFormat(JSON) # retourner le résultat au format json
results = sparql.query().convert() # stocker les résultats convertis dans une variable

date_format = "%Y-%m-%dT%H:%M:%SZ"

rows = results['results']['bindings'] # créer une liste de résultats avec le lien associé à chaque label
print(f"\n{len(rows)} Bourgmestres de la Ville de Bruxelles trouvés\n") # imprimer le nombre de politiciens recensés par la requête


for row in rows:
    try:
        starttime = dt.strptime(row['starttime']['value'], date_format) # stocker la valeur de la date de naissance au format date (string au départ)
        starttime_year = starttime.year
    except ValueError: 
        starttime_year = "????"
    except KeyError:
        starttime_year = "" 
    try:
        endtime = dt.strptime(row['endtime']['value'], date_format) 
        endtime_year = endtime.year
    except ValueError:
        endtime_year = "????"
    except KeyError:
        endtime_year = ""

    print(f"{row['personLabel']['value']} ({starttime_year}-{endtime_year})") # imprimer les 10 premiers résultats
    
# la requête saisit toutes les starttime et endtime pour toutes les fonctions politiques occupées au cours de leur vie


88 Bourgmestres de la Ville de Bruxelles trouvés

Jan-Baptist Verlooy (-)
Paul Arconati-Visconti (-)
Roger-Wauthier van der Noot, 1st Baron of Carloo (-)
Antoine de Grimbergen van Assche (-)
Adolphe Max (-)
Marion Lemesre (-)
Émile de Mot (-)
Émile de Mot (-)
Charles Buls (-)
Frederick de Marselaer (-)
Hendrik de Bloyere (-)
François-Xavier de Donnea (-)
François-Xavier de Donnea (-)
Charles de Brouckère (-)
Charles de Brouckère (-)
André-Napoléon Fontainas (-)
Wilhelmus Bloemart (-)
Wenceslas t'Serclaes (-)
Lucien Cooremans (-)
Michel Demaret (-)
Jules Coelst (-)
Gabriel van Bemmel (-)
Jules Anspach (-)
François-Jean Wyns de Raucourt (-)
Hyacinthe van der Fosse (-)
Hyacinthe van der Fosse (-)
Hyacinthe van der Fosse (-)
Yvan Mayeur (-)
Yvan Mayeur (-)
Joseph Van De Meulebroeck (-)
Jean de Locquenghien (-)
Lancelot Schetz (-)
Henri Joseph Van Langhenhoven (-)
Nicolas-Jean Rouppe (-)
Hervé Brouhon (-)
Henri Taye (1437-1438)
Adrien de Grimberghe, seigneur d'Assche (1488-1489)
John II of

In [ ]:
## Pour en savoir plus

- Le projet Wikidata : https://www.wikidata.org/wiki/Wikidata:Main_Page
- Aide à la construction de requêtes : https://query.wikidata.org/
- Exemples de requêtes : https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples/fr
- Qualifiers, references & ranks : https://en.wikibooks.org/wiki/SPARQL/WIKIDATA_Qualifiers,_References_and_Ranks

In [151]:
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" # wikidata au format RDF
sparql = SPARQLWrapper(endpoint) # stockage de l'adressage dans une variable

# P39 = fonction occupée par une personne
# Q33126365 = bourgmestre de Bruxelles
# P580 = date de début
# P582 = date de fin

statement = """
SELECT DISTINCT ?person ?personLabel ?starttime ?endtime
WHERE 
{
    ?person wdt:P39 wd:Q33126365 .
    ?person p:P39 [ps:P39 wd:Q33126365;
                   pq:P580 ?starttime;
                   pq:P582 ?endtime].
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
ORDER BY ?starttime
"""

sparql.setQuery(statement) # adresse la requête à wikidata
sparql.setReturnFormat(JSON) # retourner le résultat au format json
results = sparql.query().convert() # stocker les résultats convertis dans une variable

date_format = "%Y-%m-%dT%H:%M:%SZ"

rows = results['results']['bindings'] # créer une liste de résultats avec le lien associé à chaque label
print(f"\n{len(rows)} Bourgmestres de la Ville de Bruxelles trouvés\n") # imprimer le nombre de politiciens recensés par la requête

for row in rows:
    try:
        starttime = dt.strptime(row['starttime']['value'], date_format) # stocker la valeur de la date de naissance au format date (string au départ)
        starttime_year = starttime.year
    except ValueError: 
        starttime_year = "????"
    except KeyError:
        starttime_year = "" 
    try:
        endtime = dt.strptime(row['endtime']['value'], date_format) 
        endtime_year = endtime.year
    except ValueError:
        endtime_year = "????"
    except KeyError:
        endtime_year = ""

    print(f"{row['personLabel']['value']} ({starttime_year}-{endtime_year})") # imprimer les 10 premiers résultats
    
# il en manque qq'uns mais la requête semble correcte ; cela doit provenir de la qualité des données à la source ; date manquante à la source.
# après vérification, la liste est satisfaisante pour les dates à considérer pour les bulletins (manque qq "suppléants" seulement).


35 Bourgmestres de la Ville de Bruxelles trouvés

Henri Taye (1437-1438)
Adrien de Grimberghe, seigneur d'Assche (1488-1489)
Jacques Taye (1574-1576)
Jacques Taye (1579-1580)
Jacques Taye (1585-1586)
Lancelot I Schetz (1586-1587)
Engelbert Taie (1620-1624)
Norbert François Charles van Assche (1731-1733)
Nicolas-Jean Rouppe (1800-1802)
Louis Devos (1805-1805)
Comte Guillaume de Merode, 5th Prince de Rubempré et d'Everberghe (1805-1809)
Charles-Joseph, 4th Duke d'Ursel (1810-1814)
Joseph van der Linden d'Hooghvorst (1814-1815)
Louis de Wellens van ten Meulenberg (1815-1817)
Hyacinthe van der Fosse (1817-1820)
Louis de Wellens van ten Meulenberg (1820-1830)
Nicolas-Jean Rouppe (1830-1838)
Guillaume Van Volxem (1838-1841)
François-Jean Wyns de Raucourt (1841-1848)
Charles de Brouckère (1848-1860)
André-Napoléon Fontainas (1860-1863)
Jules Anspach (1863-1879)
Felix Vanderstraeten (1879-1881)
Charles Buls (1881-1899)
Émile de Mot (1899-1909)
Adolphe Max (1909-1939)
Joseph Van De Meulebroeck